# 기존 HAN 모델을 사용할 경우 실행

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [2]:
import pandas as pd
import numpy as np
import pickle
from eunjeon import Mecab
import os
import collections
import re
from tqdm import tqdm
import nltk

# 다른 형태소 분석기를 사용하게 될 경우 수정 필요
mecab = Mecab(dicpath = 'C:\\mecab\\mecab-ko-dic') ## dic 반영 형태소 분석기  

from collections import defaultdict
from collections import Counter 
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Activation 
from keras.layers import Conv1D, MaxPooling1D, merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.models import Model 

from keras import optimizers
from keras import backend as K
from tensorflow.keras.layers import Layer,InputSpec
from keras import initializers as initializers, regularizers, constraints

import json

from sklearn.metrics import precision_recall_curve
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

from keras.models import load_model
from datetime import datetime

import timeit

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
version_name=str(datetime.today().strftime("%Y%m%d")) # 버전 정보(오늘의 날짜) 담는 변수

# Part 2. Classification with trained HAN
- 학습된 분류모델, HAN을 가지고 분류하고 싶은 NTIS 혹은 NEWS의 ICT 분류 및 결과 저장

In [3]:
os.chdir("C:\\Users\\newcomer02\\NTIS_Project")

In [4]:
han_ver = input("\n불러올 HAN모델 폴더명을 입력하세요 (Default : 엔터키): ")
# data 폴더에 존재
ntis_ver = input("\n불러올 데이터 파일명을 입력하세요 (Default : 엔터키): ")
# Trained_HAN_kisau_20221021_0.8773
# GOV,NKIS,NTIS,REPORT


불러올 HAN모델 폴더명을 입력하세요 (Default : 엔터키):  Trained_HAN_kisau_20221021_0.8773

불러올 데이터 파일명을 입력하세요 (Default : 엔터키):  NKIS


In [204]:
# input : model 폴더명, 없으면 오류 발생 (원하는 버전 모델 있으면 지정 필요)
# 학습 모델 및 threshold 로드 (threshold 폴더 이름에 자동 저장)

if han_ver =='':
    model = load_model('./model/Default/HAN_0.4240')
    thr = float(0.4240)
    print('\nThreshold of the model : {}'.format(thr)) 
    
else:
    model = load_model('./model/'+han_ver)
    thr = float(han_ver[-6:])
    print('\nThreshold of the model : {}'.format(thr)) 
    
if thr >= 0.5 :
    thr = 0.5


Threshold of the model : 0.8773


## 2-1. NTIS 
### (1) 학습 모델 및 목적 데이터 로드

In [9]:
ntis["year"] =ntis["year"].replace(2022,2021)

In [13]:
len(ntis)

7451

In [12]:
ntis["year"]

2017    1500
2020    1475
2019    1446
2021    1397
2018    1385
2022     248
Name: year, dtype: int64

In [11]:
# ntis 데이터 로드
# input : pkl 파일, 없으면 오류 발생 (data 폴더)

if ntis_ver =='GOV':
    with open('./data/Default/' + ntis_ver + "/" + ntis_ver + "_2022.pkl", 'rb') as f:
        ntis = pickle.load(f)
else:
    with open('./data/Default/'+ ntis_ver + "/" + ntis_ver + "_2021.pkl",'rb') as f:
        ntis = pickle.load(f) 

### (2) 데이터 전처리

In [206]:
def nouns(text):
    answer = list()
    target = ["NNG","NNP","NNB","NR","NP","SL"]
    for i in mecab.pos(text):
        if i[1] in target:
            answer.append(i[0])
        else: 
            continue
    return answer
# nouns("ict iot ai atml automl 안녕 직생하다 모르겠다 장의사")

In [207]:
def preprocessing(text_list):
    global text
    for text in text_list:
        text = re.sub(r'\W+', ' ', text) 
        text = re.sub('o','', text)
        text = re.sub('○','', text)
        text = re.sub('ㅇ','', text)
        text = nouns(text)
    return text

def preprocessing_1(text_list):
    preprocessed_text = []
    for text in text_list:
        text = re.sub(r'\W+', ' ', text) 
        text = re.sub('o','', text)
        text = re.sub('○','', text)
        text = re.sub('ㅇ','', text)
        text = nouns(text)
        preprocessed_text.append(text)
    return preprocessed_text

In [208]:
from nltk.tokenize import LineTokenizer

line_tokenizer = LineTokenizer()

# 실행 시 다소 긴 소요 시간 발생 유의

ntis = ntis.reset_index(drop=True)
if ntis_ver =="GOV":
    ntis['content'] = ntis['contents'].apply(lambda text: [preprocessing(line_tokenizer.tokenize(i)) for i in text.split("\t")])
else :
    ntis['content'] = ntis['contents'].apply(lambda text: line_tokenizer.tokenize(text))
    ntis['content']= ntis.content.apply(preprocessing_1) 

In [209]:
# HAN 모델 학습 전 파라미터값 설정
# [문서, 문장, 단어]를 계층적으로 표현하기 위해 담을 Data Shape 구성

DIC = pd.read_csv('./data/Default/user-custom.csv')
#ICT_DIC = DIC[DIC['category']=='정보·통신']['term']
#list_ICT_DIC = list(ICT_DIC)
#list_ICT_DIC

In [210]:
# HAN 모델 학습 전 파라미터값 설정
# [문서, 문장, 단어]를 계층적으로 표현하기 위해 담을 Data Shape 구성

max_sent_length= 150 # 문장 내 최대 단어 수 
max_sentences = 30   # 문서 내 최대 문장 수 
max_words_dic = 25000
embedding_dim = 50 
adam = tf.optimizers.Adam(learning_rate=0.001) 

# 단어 단위 토큰화 작업
# input : json 형태의 파일, 해당 파일 없으면 오류 발생 (data/HAN 폴더)
tokenizer = Tokenizer(num_words=max_words_dic, lower=False, oov_token="term")
tokenizer.word_index = dict.fromkeys(DIC['term'])

In [211]:
new_dict = {}
k =1 
for v in tokenizer.word_index.keys():
    new_dict[v] = k
    k += 1

tokenizer.word_index = new_dict 

In [212]:
# 데이터 구성 작업 수행 : [문서, 문장, 단어]

data = np.zeros((len(ntis), max_sentences, max_sent_length), dtype='int32') 
documents = list(ntis.content)

for i, sentences in tqdm(enumerate(documents)):
    for j, sentence in enumerate(sentences): 
        if j < max_sentences:
            wordTokens = text_to_word_sequence(str(sentence))
            k = 0 
            for _, word in enumerate(wordTokens):
                try:
                    word = word.strip('\'')
                    if k < max_sent_length :
                        if word not in tokenizer.word_index:
                            continue
                        else :
                            data[i,j,k] = tokenizer.word_index[word]
                            k+=1
                    
                except SyntaxError:
                    print(word)
                    if k < max_sent_length :
                        if word not in tokenizer.word_index:
                            continue
                        else :
                            data[i,j,k] = tokenizer.word_index[word]
                            k+=1
  

910it [00:00, 2645.40it/s]


### (3)  ICT 분류
#### model input으로 NTIS 데이터를 넣어 ICT 분류 여부를 확인함 

In [213]:
# 데이터 크기에 따라 소요 시간이 커짐

s = timeit.default_timer()

Ict_prob = model.predict(data)
ntis['ICT_prob'] = Ict_prob[:,1]
ntis = ntis.sort_values(by='ICT_prob',ascending=False)
ntis['filtered_ICT'] = ntis.ICT_prob.apply(lambda x: 1 if x > thr else 0)
ntis
    
e = timeit.default_timer()
print(f"{e-s} secs taken")

5.326583899999605 secs taken


In [214]:
# ICT / Non ICT 분류 결과 확인 (filtered_ICT 컬럼)

predicted_ntis = ntis.copy()
predicted_ntis

,NUM,NAME_MAG,NAME_ART,type,M_author,J_author,keyword,year,contents,content,ICT_prob,filtered_ICT
306,4311435,AI TREND WATCH,클라우드 기반 AI에 대한 엣지 AI의 도전과 영향,"정보화,IT산업·시장",김민진,NaN,"인공지능,AI",2020,AI 알고리즘이 대량의 데이터를 처리하고 다량의 컴퓨팅 리소스를 소비함에 따라 소프...,"[[AI 알고리즘, 대량의 데이터, 처리, 다량, 의, 컴퓨팅, 리소스, 소비, 소...",0.999405,1
862,3934577,정보통신방송정책,음성인식 AI 비서 시장의 현황과 시사점,IT산업·시장,최지혜,이선희,NaN,2017,스마트폰에 시작된 음성인식 기술 경쟁은 사물인터넷 IoT 의 본격 확산과 맞물려 이...,"[[스마트폰, 시작, 음성, 인식 기술, 경쟁, 은, 사물인터넷, IT, 본격, 확...",0.999074,1
92,4333447,KISDI STAT Report,개인방송 이용 행태 및 이용자 특성 분석,방송콘텐츠,이선희,NaN,NaN,2021,2000년대 중후반부터 사용자가 직접 창작한 미디어 콘텐츠 형태인 UCC Us...,"[[중후반, 사용, 자가, 직접, 창작, 미디어, 콘텐츠, 형태, UCC, User...",0.999033,1
824,3934577,정보통신방송정책,IoT 생태계 확산과 엣지 컴퓨팅의 역할,IT산업·시장,손가녕,이은민,NaN,2017,IoT생태계가 확산됨에 따라 무수히 많은 다양한 IoT사물에서 빅데이터가 발생될 것...,"[[IT, 생태, 계가, 확산, IT, 사물, 빅데이터, 가, 발생, 것, 예상, ...",0.998959,1
253,4311435,AI TREND WATCH,모바일 클라우드 컴퓨팅 기반 연합학습의 발전 및 최신 경향,"정보화,IT산업·시장",이진우,NaN,NaN,2020,모바일 클라우드 컴퓨팅은 모바일 기기의 급격한 발달과 전 세계적인 확산으로 인해 ...,"[[모바일 클라우드 컴퓨팅, 은, 모바일 기기, 의, 급격, 세계, 확산, 사용],...",0.998851,1
...,...,...,...,...,...,...,...,...,...,...,...,...
873,3934577,정보통신방송정책,2016 ICT벤처기업 현황분석: 벤처기업정밀실태조사 결과를 중심으로,IT산업·시장,이은민,NaN,NaN,2017,중소기업청 8228 벤처기업협회는 매년 벤처기업정밀실태조사 보고서를 발간하여 ...,"[[중소기업청, 벤처기업, 협회, 매년, 벤처기업, 정밀, 실태조사, 벤처기업, 의...",0.005076,0
6,4333447,KISDI STAT Report,세대별 OTT 서비스 이용 현황,"방송정책,방송콘텐츠",김윤화,NaN,"한국미디어패널, OTT이용율, OTT 사용 시간, Z세대",2022,OTT 이용률은 2019년 41 0 2020년 72 2 2021년 81 ...,"[[OTT, 이용률, 은, 로, 이후, 코로나, 이용, 증가, 추세], [코로나, ...",0.004828,0
91,4334696,KISDI Perspectives,[초점]우리나라 ICT 수출의 구조적 단절과 시사점,"IT산업·시장,통상",고동환,NaN,"ICT 수출,구조적 단절,오차수정모형",2021,2000년 이후 ICT 분야에서 중국은 세계 공급망의 중심으로 부상하였으나 내...,"[[이후, ICT, 분야, 중국, 은, 세계, 공급, 망의, 중심, 부상, 내부, ...",0.004264,0
839,3934550,KISDI Premium Report,4차 산업혁명과 지능정보사회의 규범 재정립,NaN,이원태,NaN,NaN,2017,본고의 목적은 지능정보사회의 규범적 대응과 관련된 지금까지의 국내외 논의 및 관련 ...,"[[고의, 지능정보사회, 의, 규범적, 응과, 관련, 지금, 국내외, 논의, 및, ...",0.003719,0


In [215]:
predicted_ntis["filtered_ICT"].value_counts()

1    632
0    278
Name: filtered_ICT, dtype: int64

In [216]:
predicted_ntis.to_excel(".\\data\\Default\\"+ ntis_ver + "/HAN_Filtered_"+ntis_ver+  ".xlsx")

### (4) 분류 결과 저장 및 갱신
- Classification 결과 ICT 여부 컬럼을 함께 저장 (news의 경우 ICT 뉴스만 남기고 저장) 

In [217]:
# output type : data 폴더에 pkl 형식으로 변환 (사용자명, 오늘 날짜)

predicted_ntis.to_pickle('./data/Default/' + ntis_ver + "/HAN_Filtered_" + ntis_ver +'.pkl')

In [218]:
pred_data = pd.read_pickle("./data/Default/" + ntis_ver + "/HAN_Filtered_"+ntis_ver+".pkl")